In [2]:
%matplotlib inline

In [3]:
import os
import json
import re
import random

In [4]:
def getJobFeature(fn_num=5):
    bdir = "./ucan/"
    pick_files = random.sample(os.listdir(bdir), fn_num)
    cnts = [ open(bdir+pick_file, 'r', encoding='utf8').read() for pick_file in pick_files

SyntaxError: unexpected EOF while parsing (<ipython-input-4-b11f137ac110>, line 4)

In [5]:
bdir = "./jobs/"
cnts = [ json.loads(open(bdir+fn, 'r', encoding='utf8').read()) for fn in os.listdir(bdir)]

# 初步整理 pay_type

In [ ]:
print (cnts[0]['pay_type'])

In [ ]:
import collections

In [ ]:
def count_pay_avg(pay_type):
    if re.search(u"[時|月|日]薪", pay_type):
        pay = pay_type.replace("時薪", '').replace("月薪", '').replace("日薪", '')
        (pay_up, pay_down) = pay.strip().split(" - ")
        pay_avg = (int(pay_up) + int(pay_down) )/2
        return pay_avg
    else:
        return pay_type

In [ ]:
pay_types = collections.Counter([ count_pay_avg(x['pay_type']) for x in cnts])

In [ ]:
import pandas as pd

In [ ]:
pay_type_df = pd.DataFrame(pay_types.most_common(1000))

In [ ]:
pay_type_df.columns = ['pay_mount', 'counter']

In [ ]:
pay_type_df

In [ ]:
pay_type_df[(pay_type_df.counter < 500) & 
            (pay_type_df.counter > 50)].plot(x='pay_mount', y='counter')

# 準備 job_desc 的資料(x)

In [ ]:
desc = [ x['ucan_desc'] for x in cnts]

In [ ]:
print (desc[0])

In [ ]:
import jieba

In [ ]:
def token(txt):
    wrds = []
    for wrd in jieba.cut(txt):
        if re.match(u"[\u4e00-\u9fa5]+", wrd): #中文字
            wrds.append(wrd)
#         elif re.match("\d+", wrd): # 數字
#             wrds.append(wrd)
#         elif re.match("\w+", wrd): # 英文字
#             wrds.append(wrd)
        else:
            pass
    return wrds

In [ ]:
print ( token(desc[0]) )

## 使用 TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(lowercase=False, 
                        sublinear_tf = True, 
                        max_df = 0.85, 
                        analyzer=lambda x: x, 
                        token_pattern=None) 

In [ ]:
ans = tfidf.fit_transform([token(x) for x in desc])

In [ ]:
feature_words = tfidf.get_feature_names()

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    return zip(feature_vals, score_vals)

In [ ]:
feature_names=tfidf.get_feature_names()

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(ans.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,100)
 
# now print the results
print("\n===Keywords===")
for (w, s) in keywords:
    print("%s=%.2f"%(w, s))

## using Kmeans

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=200, random_state=0)

In [ ]:
kmeans = kmeans.fit(ans) # ans TFIDF 向量化的結果

In [ ]:
kmeans.labels_.shape

In [ ]:
print (ans.shape, len(desc))

In [ ]:
ans[0]

In [ ]:
desc[0]

In [ ]:
label_dic = {}
for idx in range(len(desc)):
    kmean_label = kmeans.labels_[idx]
    if not kmean_label in label_dic:
        label_dic[ kmean_label ] = []
    label_dic[ kmean_label ].append(desc[idx])


In [ ]:
print (label_dic.keys())

In [ ]:
for lbl in label_dic.keys():
    print ("-"*10, lbl, "(%s)"%(len(label_dic[lbl])), "-"*10)
    print ("\n".join(label_dic[lbl][:3]))

## install jieba

In [ ]:
!pip install jieba

# install dill

In [ ]:
!pip install dill

In [ ]:
import dill

In [ ]:
dill.dump(tfidf, open("model_tfidf.dil", 'wb'))

In [ ]:
dill.dump(kmeans, open("model_kmeans.dil", 'wb'))